In [27]:
pip install -r requirements.txt

Processing c:\home\conda\feedstock_root\build_artifacts\appnope_1649077682618\work (from -r requirements.txt (line 3))
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\home\\conda\\feedstock_root\\build_artifacts\\appnope_1649077682618\\work'



In [28]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [29]:
from openai import OpenAI

client = OpenAI()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4-1106-preview",
)
chat_completion.choices[0].message

ChatCompletionMessage(content='This is a test.', role='assistant', function_call=None, tool_calls=None)

In [30]:
from app.llm.utils import schema_to_openai_func


def _new_file(full_path: str) -> str:
    """
    Create a new file at the given path. Returns "success" if successful, otherwise an error message.
    """
    pass

def _remove_lines(start: int, end: int) -> str:
    """
    Remove the lines between the given start and end line numbers. Returns "success" if successful, otherwise an error message.
    """
    
def _insert_lines(at: int, append_content: str) -> str:
    """
    Insert the given content at the given line number. Returns "success" if successful, otherwise an error message.
    """
    
def _replace_lines(start: int, end: int, new_content: str) -> str:
    """
    Replace the lines between the given start and end line numbers with the given content. Returns "success" if successful, otherwise an error message.
    """

def _execute_command(command: str) -> str:
    """
    Execute the given command in the integrated terminal. Returns the output of the command.
    """

    
from pydantic import BaseModel, Field

class FakeProject:
    def __init__(self):
        self.files = {} # {full_path: content}
        self.current_file = None # set by open_file

    def new_file(self, full_path: str):
        if full_path in self.files:
            return f"File already exists at {full_path}"
        self.files[full_path] = ""
        return "success"
    
    def open_file(self, full_path: str):
        if full_path not in self.files:
            return f"File does not exist at {full_path}"
        self.current_file = full_path
        return "success"
    
    def remove_lines(self, start: int, end: int):
        if start > end:
            return "Start line cannot be greater than end line."
        if not self.current_file:
            return "No file is currently open, please open a file first using the open_file tool."
        
        lines = self.files[self.current_file].split("\n")
        if start > len(lines):
            return "Start line is greater than the number of lines in the file."
        if end > len(lines):
            return "End line is greater than the number of lines in the file."
        
        self.files[self.current_file] = "\n".join(lines[:start] + lines[end:])
        return "success"

class remove_lines(BaseModel):
    """
    Remove the lines between the given start and end line numbers. Returns "success" if successful, otherwise an error message.
    """
    start: int = Field(description="The line number to start removing at.")
    end: int = Field(description="The line number to stop removing at.")

class insert_lines(BaseModel):
    """
    Insert the given content at the given line number. Returns "success" if successful, otherwise an error message.
    """
    at: int = Field(description="The line number to insert at.")
    append_content: str = Field(description="The content to insert.")

class replace_lines(BaseModel):
    """
    Replace the lines between the given start and end line numbers with the given content. Returns "success" if successful, otherwise an error message.
    """
    start: int = Field(description="The line number to start replacing at.")
    end: int = Field(description="The line number to stop replacing at.")
    new_content: str = Field(description="The content to replace with.")

class new_file(BaseModel):
    """
    Create a new file at the given path. Returns "success" if successful, otherwise an error message.
    """
    full_path: str = Field(description="The full path of the file to create.")
    

class execute_command(BaseModel):
    """
    Execute the given command in the integrated terminal. Returns the output of the command.
    """
    command: str = Field(description="The command to execute.")


In [31]:
from openai import OpenAI
client = OpenAI()

SYSTEM = """
You are an AI copilot directly integrated into an IDE. You have full access to the interface of the IDE by calling the provided function tools, such as creating new files, writing to files, and executing commands on the integrated terminal.
When the user requests something, you should call a sequence of appropriate functions to fulfill the request.
For example, when the user requests "make and execute a simple hello world python program", you should call the following functions:
1. new_file("src/main.py")
2. insert_lines(1, "print('hello world')")
3. execute_command("python src/main.py")
"""

tools = [
  schema_to_openai_func(new_file),
  schema_to_openai_func(remove_lines),
  schema_to_openai_func(execute_command),
  schema_to_openai_func(insert_lines),
  schema_to_openai_func(replace_lines)
]
messages = [
    {"role": "system", "content": SYSTEM},
    {"role": "user", "content": "make and execute a simple hello world python program"}
  ]
completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=messages,
  tools=tools,
  tool_choice="auto",
  temperature=0.0,
)

my_project = FakeProject()

for tool_call in completion.choices[0].message.tool_calls:
    f = tool_call.function
    print(f)
    if f.name == 'new_file':
        full_path = f.arguments["full_path"]
        ret_val = my_project.new_file(full_path)

        # insert the return value into the message
        messages.append({"role": "function", "content": ret_val})


Function(arguments='{"full_path": "src/main.py"}', name='new_file')
Function(arguments='{"at": 1, "append_content": "print(\'hello world\')"}', name='insert_lines')


In [32]:
prompt = """

```python
from fastapi import APIRouter
from app.services import hello_service

router = APIRouter()


@router.get("/test")
def hello_controller(query_string: str):
    result = hello_service.hello(query_string)
    return result

@router.get("/user_action")
def user_action_controller(query_string: str):
    result = hello_service.user_action(query_string)
    return result

<CURSOR>
def ai_action_controller(query_string: str):
    pass
```

Where <CURSOR> denotes the current postion of the cursor. Any code you write using insert_text function tool will insert that content in the place of the cursor.

Turn the ai_action_controller into an endpoint.
"""